In [1]:
!pip3 install hdbscan 
!pip3 install bertopic

In [2]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
from tqdm import tqdm 
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/express/imshow_utils.py:24: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/

In [34]:
red_states = pd.read_csv('/Users/lydiafield/Desktop/Semestre 1/Digital Methods/red_states.csv')

In [36]:
docs=list(red_states['Hit Sentence'])
model = SentenceTransformer("all-MiniLM-L6-v2")
document_vectors = model.encode(docs,show_progress_bar=True)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.01, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=5, metric='euclidean', cluster_selection_method='eom')
vectorizer = CountVectorizer(stop_words='english')
topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    verbose=True
).fit(docs, document_vectors)
bar_topic = topic_model.visualize_barchart(top_n_topics=20,n_words=10, title='Red States Topics')

Batches:   0%|          | 0/198 [00:00<?, ?it/s]

2023-12-12 01:56:44,947 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-12 01:56:51,920 - BERTopic - Dimensionality - Completed ✓
2023-12-12 01:56:51,923 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-12 01:56:52,041 - BERTopic - Cluster - Completed ✓
2023-12-12 01:56:52,046 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-12 01:56:52,254 - BERTopic - Representation - Completed ✓


In [37]:
bar_topic

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [38]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2402,-1_book_books_desantis_banning,"[book, books, desantis, banning, ban, banned, ...",[🧵Moms for Liberty and other far right so-call...
1,0,198,0_queer_explicit_porn_sexually,"[queer, explicit, porn, sexually, children, ha...",[ @ChelseaClinton: If questioning the age-appr...
2,1,110,1_insurance_florida_state_lgbtq,"[insurance, florida, state, lgbtq, expand, say...",[ @equalityfl: Fascist DeSantis and his puppet...
3,2,103,2_black_white_history_racism,"[black, white, history, racism, supremacist, l...",[ @PoetTonyMedina: That a school board would a...
4,3,102,3_pink_away_concerts_stops,"[pink, away, concerts, stops, singer, 000, tou...",[ @CalltoActivism: Good Idea....Pink & Swift a...
...,...,...,...,...,...
133,132,10,132_whitmer_interested_mollyploofkins_invade,"[whitmer, interested, mollyploofkins, invade, ...","[Ron DeSantis bans books, bans history, wants ..."
134,133,10,133_instances_prevalent_charted_reidout,"[instances, prevalent, charted, reidout, blog,...","[Nearly 1,500 instances of book-banning were c..."
135,134,10,134_cuban_celia_cruz_castro,"[cuban, celia, cruz, castro, carmenpelaez, fid...","[ @DebbieforFL: Banning books, hobbling educat..."
136,135,10,135_57_math_1100_129,"[57, math, 1100, 129, crt, result, objected, p...",[ @MarvinDunn4: Good question Remember-all the...


In [17]:
import pandas as pd
import tqdm

In [39]:
corpus_text = red_states['Hit Sentence'].values

In [40]:
nlp = spacy.load("en_core_web_sm")
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)

  0%|                                                  | 0/6328 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████████████████████████████████| 6328/6328 [00:23<00:00, 271.21it/s]


In [41]:
docs2 = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]
docs2 = [[token for token in doc if len(token) > 3] for doc in docs2]

In [42]:
id2word = Dictionary(docs2)
id2word.filter_extremes(no_below=20, no_above=0.1)

In [43]:
corpus = [id2word.doc2bow(text) for text in docs2]

In [44]:
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   passes=20)

In [45]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 free, kill, speech, every, america, fascist, sure, until, fascism, trump
1 :	 child, your, should, time, would, teacher, explicit, keep, thing, happen
2 :	 district, child, read, remove, parent, year, which, student, list, when
3 :	 your, liberty, anti, group, hate, moms, back, lgbtq, take, board
4 :	 county, week, texas, shelf, judge, public, more, just, order, after
5 :	 bible, parent, read, black, utah, race, books, challenge, person, than
6 :	 education, over, district, student, gender, freedom, public, think, house, lgbtq
7 :	 away, give, make, black, child, because, pink, america, teach, enough
8 :	 republicans, vote, public, teacher, bill, child, republican, just, pass, then
9 :	 county, nazi, control, more, duval, republican, abortion, list, governor, other


In [46]:
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

-3.719364160531149

In [47]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.163075 -0.027480       1        1   8.996633
1      0.035398 -0.167938       2        1  10.324837
2     -0.082692 -0.021011       3        1  11.633800
3      0.122967 -0.054453       4        1  12.635213
4     -0.308480 -0.121366       5        1   8.732031
5     -0.038185  0.258057       6        1   8.010312
6      0.052109  0.073119       7        1  10.241186
7      0.105261 -0.070394       8        1   8.686594
8      0.020831  0.007011       9        1  12.760640
9     -0.070283  0.124455      10        1   7.978754, topic_info=            Term        Freq       Total Category  logprob  loglift
210       county  575.000000  575.000000  Default  30.0000  30.0000
72          your  688.000000  688.000000  Default  29.0000  29.0000
236     district  451.000000  451.000000  Default  28.0000  28.0000
21   republicans  445.000000  445.000000  Default  27.0000  27.0000
451      liberty  287.000000  287.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
83          here   62.582530  358.969346  Topic10  -4.7059   0.7816
179        fight   61.641978  334.582344  Topic10  -4.7211   0.8368
51       against   58.814868  387.618882  Topic10  -4.7680   0.6428
264      because   58.467471  434.422206  Topic10  -4.7739   0.5228
66          make   57.325240  572.040134  Topic10  -4.7937   0.2279

[636 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
745      10  0.980403     aaron
987      10  0.980528    abbott
126       4  0.300194  abortion
126       7  0.016865  abortion
126       9  0.418247  abortion
...     ...       ...       ...
72        2  0.248482      your
72        3  0.007266      your
72        4  0.662618      your
72        5  0.015984      your
72        8  0.062484      your

[1263 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [48]:
docs_2_topics = lda_model[corpus]
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [49]:
topic_number=9
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

county, nazi, control, more, duval, republican, abortion, list, governor, other
DeSantis' campaign promises include sticking it to the libs. He promises to chase woke culture and rid the world of it. (he doesn't even know what woke is). He promises to ban books and art if it makes a racist bigot uncomfortable. (white discomfort) He promises to exclude…   /2uIcy1mIKA 0.9470508
Don't be fooled - No Labels supports DeSantis as a "moderate" alternative. There's no world where a person who bans books, kidnaps immigrants, & attacks businesses is the moderate one. No Labels is MAGA, full stop. Listen to @TaraSetmayer and @TheRickWilson break it down below.   /3ADLZIkoTU 0.94704443
ICYMI - No Labels supports DeSantis as a "moderate" alternative. There's no world where a person who bans books, kidnaps immigrants, and attacks businesses is the moderate one. No Labels is MAGA, full stop. Listen to @TaraSetmayer and @TheRickWilson break it down below.   /1LJRvP3ruW 0.94704443
The jig is up - No La